<a href="https://colab.research.google.com/github/quasistellar0905/iTshirt2/blob/master/Lec4_pydeseq2_CD_tutorial_answer_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyDESeq2 실습
tutorial은 PyDESeq2 최신 문서를 참고하였습니다. (링크: https://pydeseq2.readthedocs.io/en/latest/index.html)






## PyDESeq2 설치
pip를 통해 PyDESeq2 라이브러리를 설치합니다

In [ ]:
!pip install pydeseq2

## 간단한 PyDESeq2 워크플로우 실습

PyDESeq2를 이용하여 bulk RNA-seq 샘플로 differential expression analysis (DEA)를 하는 방법을 알아보겠습니다.

우선 필요한 library를 import합니다

In [ ]:
from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from pydeseq2.utils import load_example_data

import pandas as pd

## 데이터 로딩
DEA을 수행하기 위해, PyDESeq2는 두 종류의 입력을 필요로 합니다:
1. 카운트데이터 행렬: read 카운트 (양의 정수)를 포함하는 '샘플 수' x '유전자 수'의 형태
2. 메타데이터 행렬: 데이터를 코호트로 분할하는 데 사용될 샘플 주석을 포함하는 '샘플 수' x '변수 수'의 형태

둘 다 pandas 데이터프레임으로 제공되어야 합니다.



---


데이터를 다운받고, 압축을 풀고, 파일을 구글코랩에 업로드합니다.

예제 데이터 다운로드 링크: https://drive.google.com/file/d/1Olc2FvdGlJC2G1SwHjb5DstD3ozlB11b/view?usp=sharing

본 예제에서는 [GSE70466](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE70466) 데이터를 이용합니다.



In [ ]:
counts_df = pd.read_csv("./GSE70466-expression.txt", sep="\t", index_col=0)
metadata = pd.read_csv("./GSE70466-metadata.txt", sep="\t", index_col=0)

In [ ]:
display(counts_df)

In [ ]:
display(metadata)

이 예제에서 metadata는 cell line에 대한 정보를 담고있습니다. normal prostate cell인 PrEC를 control샘플로, prostate cancer cell인 LNCaP를 case샘플로 다룹니다.

## 데이터 필터링
DEA를 진행하기 전에 데이터 전처리를 위하여 표현 수준이 매우 낮은 유전자를 제외합니다. 총 10개 미만의 읽기 카운트를 가진 유전자를 필터링합니다


In [ ]:
genes_to_keep = counts_df.columns[counts_df.sum(axis=0) >= 10]
counts_df = counts_df[genes_to_keep]

In [ ]:
display(counts_df)

## DeseqDataSet 클래스를 사용한 read 카운트 모델링
우리는 카운트와 메타데이터로부터 DeseqDataSet 객체를 생성하여 시작합니다. DeseqDataSet는 데이터로부터 dispersion 및 log2 fold Change (LFC)를 fitting하고 저장합니다.






In [ ]:
dds = DeseqDataSet(
    counts=counts_df,
    metadata=metadata,
    design_factors="cell line",
    ref_level=["cell line", "PrEC"],
    refit_cooks=True,
    n_cpus=8,
)

design_factors는 샘플 class 정보가 포함된 metadata 내의 column 이름을 의미합니다.

ref_level은 design_factor중 control group을 정의합니다.

refit_cooks 인수(기본적으로 True로 설정됨)는 Cooks 이상치가 재적합되어야 하는지를 제어합니다 (일반적으로 권장됨).

DeseqDataSet가 초기화되면, 우리는 deseq2() 메서드를 실행하여 dispersion과 LFC를 적합시킬 수 있습니다.

In [ ]:
dds.deseq2()

참고로 DeseqDataSet 클래스는 AnnData 클래스를 상속합니다

In [ ]:
print(dds)

따라서, 매개변수는 키 기반 데이터 필드를 사용하여 AnnData 데이터 구조에 따라 저장됩니다. 특히,

X는 카운트 데이터를 저장합니다,
obs는 디자인 요인을 저장합니다,
obsm은 "design_matrix"와 "size_factors"와 같은 샘플 수준의 데이터를 저장합니다,
varm은 "dispersions"와 "LFC"와 같은 유전자 수준의 데이터를 저장합니다.
예를 들어, 우리가 dispersion과 LFC에 접근하는 방법은 다음과 같습니다:

In [ ]:
print(dds.varm["dispersions"])

In [ ]:
display(dds.varm["LFC"])

## DeseqStats 클래스를 사용한 통계 분석
이제 dispersion과 LFC가 fitting 되었으므로, 차등 발현에 대한 p value와 adjusted p value를 계산하기 위한 통계 검정을 진행할 수 있습니다.

In [ ]:
stat_res = DeseqStats(dds, alpha=0.05, n_cpus=8)

optional argument들은 아래와 같습니다.

alpha: pvalue 및 adjusted pvalue의 유의성 임계값 (기본값은 0.05),

cooks_filter: cooks 이상치를 기반으로 p-값을 필터링할지 여부 (기본값은 True),

independent_filter: p value추세를 수정하기 위해 독립적인 필터링을 수행할지 여부 (기본값은 True).

## Wald 검정
PyDESeq2는 Wald 검정을 사용하여 p value를 계산합니다. 이것은 summary() 메서드를 사용하여 수행될 수 있으며, 이 메서드는 전체 통계 분석, Cooks 필터링 및 다중 검정 조정을 포함하여 실행됩니다.

In [ ]:
stat_res.summary()

adjusted pvalue를 기준으로 유의한 gene들을 추려내고, LFC로 sorting합니다

In [ ]:
deseq_results = stat_res.results_df[stat_res.results_df["padj"]<0.05].sort_values("log2FoldChange", ascending=False)
display(deseq_results)

## 상위 500개 gene 추출

case sample에서 over expressed된 상위 500개의 gene과 under expressed된 하위 500개 gene을 추출합니다

In [ ]:
up_genes = deseq_results.index[:500].tolist()

In [ ]:
down_genes = deseq_results.index[-500:].tolist()

# Characteristic Direction (CD) 실습

## Characteristic Direction (CD) 설치
pip를 통해 characteristic direction 라이브러리를 설치합니다

In [ ]:
!pip install git+https://github.com/Maayanlab/maayanlab-bioinformatics.git

필요한 library를 import합니다

In [ ]:
from maayanlab_bioinformatics.dge.characteristic_direction import characteristic_direction

# 데이터 전처리
CD는 normalized 된 count 데이터를 입력으로 받으며, case matrix와 control matrix를 따로 input으로 받습니다. 각 matrix는 '유전자의 수' x '샘플 수'의 형태입니다.

In [ ]:
normalized_counts_df = pd.DataFrame(dds.layers["normed_counts"], columns=dds.var.index.tolist(), index=dds.obs.index.tolist())

In [ ]:
ctrl_metadata = dds.obs[dds.obs["cell line"]=="PrEC"]
case_metadata = dds.obs[dds.obs["cell line"]!="PrEC"]

In [ ]:
ctrl_normalized_counts_df = normalized_counts_df.loc[ctrl_metadata.index,:].T
case_normalized_counts_df = normalized_counts_df.loc[case_metadata.index,:].T

## CD 계산
첫번째 인자로 control 샘플에 대한 normalized count 데이터를, 두번째 인자로는 case 샘플에 대한 normalized count 데이터를 입력합니다. significance를 계산하기 위해 calculate_sig를 True로 합니다.

In [ ]:
cd_results = characteristic_direction(controls_mat=ctrl_normalized_counts_df, cases_mat=case_normalized_counts_df, calculate_sig=True)
display(cd_results)

## 유의미한 유전자 추출

In [ ]:
cd_results = cd_results[cd_results["Significance"]<0.05].sort_values("CD-coefficient", ascending=False)
display(cd_results)

In [ ]:
up_genes = cd_results.index[:500].tolist()

In [ ]:
down_genes = cd_results.index[-500:].tolist()